In [142]:
# Dependencies
import requests
from config_env import api_key
import time
import json
import os
from IPython.core.display import display, HTML
import pymongo
from pymongo import MongoClient
#for scraping article
from bs4 import BeautifulSoup as bs


In [14]:
url = "https://api.nytimes.com/svc/search/v2/articlesearch.json?"

# Store a search term
query = "Venezuela"
param = 'fq=news_desk:(Foreign)&fq=source:(The New York Times)'
# Search for articles published between a begin and end date
begin_date = "20140101"
end_date = "20190101"
page = '&page='

query_url = f"{url}{param}&api-key={api_key}&q={query}&\
facet_field=snippet&begin_date={begin_date}&end_date={end_date}{page}"

In [20]:
output_list = []
# Retrieve articles

for n in range(1, 101):
    url=query_url + str(n)
    try:
        articles = requests.get(url).json()
        print(f"Getting data from {url}")
        articles_list = [article for article in articles["response"]["docs"]]
    
        for item in articles_list:
            list_dict = {}
            list_dict["snippet"] = item["snippet"]
            list_dict["lede"] = item["lead_paragraph"]
            list_dict["url"] = item["web_url"]
            list_dict["headline"] = item["headline"]["main"]
            list_dict["keywords"] = item["keywords"]
            list_dict["pub_date"] = item["pub_date"]
            output_list.append(list_dict)
        
        time.sleep(6)
    except KeyError:
        print(f"Error with {n}")
        pass



Getting data from https://api.nytimes.com/svc/search/v2/articlesearch.json?fq=news_desk:(Foreign)&fq=source:(The New York Times)&api-key=SrHpw0UozDxB5L0HPQbQXTM9n89lcikp&q=Venezuela&facet_field=snippet&begin_date=20140101&end_date=20190101&page=1
Getting data from https://api.nytimes.com/svc/search/v2/articlesearch.json?fq=news_desk:(Foreign)&fq=source:(The New York Times)&api-key=SrHpw0UozDxB5L0HPQbQXTM9n89lcikp&q=Venezuela&facet_field=snippet&begin_date=20140101&end_date=20190101&page=2
Getting data from https://api.nytimes.com/svc/search/v2/articlesearch.json?fq=news_desk:(Foreign)&fq=source:(The New York Times)&api-key=SrHpw0UozDxB5L0HPQbQXTM9n89lcikp&q=Venezuela&facet_field=snippet&begin_date=20140101&end_date=20190101&page=3
Getting data from https://api.nytimes.com/svc/search/v2/articlesearch.json?fq=news_desk:(Foreign)&fq=source:(The New York Times)&api-key=SrHpw0UozDxB5L0HPQbQXTM9n89lcikp&q=Venezuela&facet_field=snippet&begin_date=20140101&end_date=20190101&page=4
Getting data

Getting data from https://api.nytimes.com/svc/search/v2/articlesearch.json?fq=news_desk:(Foreign)&fq=source:(The New York Times)&api-key=SrHpw0UozDxB5L0HPQbQXTM9n89lcikp&q=Venezuela&facet_field=snippet&begin_date=20140101&end_date=20190101&page=35
Getting data from https://api.nytimes.com/svc/search/v2/articlesearch.json?fq=news_desk:(Foreign)&fq=source:(The New York Times)&api-key=SrHpw0UozDxB5L0HPQbQXTM9n89lcikp&q=Venezuela&facet_field=snippet&begin_date=20140101&end_date=20190101&page=36
Getting data from https://api.nytimes.com/svc/search/v2/articlesearch.json?fq=news_desk:(Foreign)&fq=source:(The New York Times)&api-key=SrHpw0UozDxB5L0HPQbQXTM9n89lcikp&q=Venezuela&facet_field=snippet&begin_date=20140101&end_date=20190101&page=37
Getting data from https://api.nytimes.com/svc/search/v2/articlesearch.json?fq=news_desk:(Foreign)&fq=source:(The New York Times)&api-key=SrHpw0UozDxB5L0HPQbQXTM9n89lcikp&q=Venezuela&facet_field=snippet&begin_date=20140101&end_date=20190101&page=38
Getting 

Getting data from https://api.nytimes.com/svc/search/v2/articlesearch.json?fq=news_desk:(Foreign)&fq=source:(The New York Times)&api-key=SrHpw0UozDxB5L0HPQbQXTM9n89lcikp&q=Venezuela&facet_field=snippet&begin_date=20140101&end_date=20190101&page=68
Error with 68
Getting data from https://api.nytimes.com/svc/search/v2/articlesearch.json?fq=news_desk:(Foreign)&fq=source:(The New York Times)&api-key=SrHpw0UozDxB5L0HPQbQXTM9n89lcikp&q=Venezuela&facet_field=snippet&begin_date=20140101&end_date=20190101&page=69
Error with 69
Getting data from https://api.nytimes.com/svc/search/v2/articlesearch.json?fq=news_desk:(Foreign)&fq=source:(The New York Times)&api-key=SrHpw0UozDxB5L0HPQbQXTM9n89lcikp&q=Venezuela&facet_field=snippet&begin_date=20140101&end_date=20190101&page=70
Error with 70
Getting data from https://api.nytimes.com/svc/search/v2/articlesearch.json?fq=news_desk:(Foreign)&fq=source:(The New York Times)&api-key=SrHpw0UozDxB5L0HPQbQXTM9n89lcikp&q=Venezuela&facet_field=snippet&begin_date=2

In [21]:
len(output_list)

532

In [135]:
print('Mongo version', pymongo.__version__)
client = MongoClient('localhost', 27017)
db = client.venezuela   

Mongo version 3.7.2


In [23]:
db.nyt_news.insert_many(output_list)

In [64]:
news = db.nyt_news.find()
news

In [65]:
# Get a list of URLs to scrape
urls = []
for item in news:
    urls.append(item['url'])

In [66]:
len(urls)

532

In [69]:
out_list = []
# Visit the NYT site and get the body copy
for item in urls:
    output_dict = {}
    response = requests.get(item, verify=False)
    soup = bs(response.text, 'lxml')
    body_copy = soup.find('div', class_='css-53u6y8')
    output_dict['body_copy'] = str(body_copy)
    db.nyt_news.update_one({'url':item}, {'$set': output_dict})
    time.sleep(2)


/Users/laura/anaconda3/envs/PythonData/lib/python3.6/site-packages/urllib3/connectionpool.py:857: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Users/laura/anaconda3/envs/PythonData/lib/python3.6/site-packages/urllib3/connectionpool.py:857: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Users/laura/anaconda3/envs/PythonData/lib/python3.6/site-packages/urllib3/connectionpool.py:857: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Users/laura/anaconda3/envs/PythonData/l

/Users/laura/anaconda3/envs/PythonData/lib/python3.6/site-packages/urllib3/connectionpool.py:857: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Users/laura/anaconda3/envs/PythonData/lib/python3.6/site-packages/urllib3/connectionpool.py:857: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Users/laura/anaconda3/envs/PythonData/lib/python3.6/site-packages/urllib3/connectionpool.py:857: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Users/laura/anaconda3/envs/PythonData/l

/Users/laura/anaconda3/envs/PythonData/lib/python3.6/site-packages/urllib3/connectionpool.py:857: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Users/laura/anaconda3/envs/PythonData/lib/python3.6/site-packages/urllib3/connectionpool.py:857: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Users/laura/anaconda3/envs/PythonData/lib/python3.6/site-packages/urllib3/connectionpool.py:857: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Users/laura/anaconda3/envs/PythonData/l

/Users/laura/anaconda3/envs/PythonData/lib/python3.6/site-packages/urllib3/connectionpool.py:857: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Users/laura/anaconda3/envs/PythonData/lib/python3.6/site-packages/urllib3/connectionpool.py:857: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Users/laura/anaconda3/envs/PythonData/lib/python3.6/site-packages/urllib3/connectionpool.py:857: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Users/laura/anaconda3/envs/PythonData/l

/Users/laura/anaconda3/envs/PythonData/lib/python3.6/site-packages/urllib3/connectionpool.py:857: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Users/laura/anaconda3/envs/PythonData/lib/python3.6/site-packages/urllib3/connectionpool.py:857: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Users/laura/anaconda3/envs/PythonData/lib/python3.6/site-packages/urllib3/connectionpool.py:857: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Users/laura/anaconda3/envs/PythonData/l

...

/Users/laura/anaconda3/envs/PythonData/lib/python3.6/site-packages/urllib3/connectionpool.py:857: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Users/laura/anaconda3/envs/PythonData/lib/python3.6/site-packages/urllib3/connectionpool.py:857: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Users/laura/anaconda3/envs/PythonData/lib/python3.6/site-packages/urllib3/connectionpool.py:857: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Users/laura/anaconda3/envs/PythonData/l

/Users/laura/anaconda3/envs/PythonData/lib/python3.6/site-packages/urllib3/connectionpool.py:857: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Users/laura/anaconda3/envs/PythonData/lib/python3.6/site-packages/urllib3/connectionpool.py:857: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Users/laura/anaconda3/envs/PythonData/lib/python3.6/site-packages/urllib3/connectionpool.py:857: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Users/laura/anaconda3/envs/PythonData/l

/Users/laura/anaconda3/envs/PythonData/lib/python3.6/site-packages/urllib3/connectionpool.py:857: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Users/laura/anaconda3/envs/PythonData/lib/python3.6/site-packages/urllib3/connectionpool.py:857: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Users/laura/anaconda3/envs/PythonData/lib/python3.6/site-packages/urllib3/connectionpool.py:857: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Users/laura/anaconda3/envs/PythonData/l

/Users/laura/anaconda3/envs/PythonData/lib/python3.6/site-packages/urllib3/connectionpool.py:857: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Users/laura/anaconda3/envs/PythonData/lib/python3.6/site-packages/urllib3/connectionpool.py:857: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Users/laura/anaconda3/envs/PythonData/lib/python3.6/site-packages/urllib3/connectionpool.py:857: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Users/laura/anaconda3/envs/PythonData/l

/Users/laura/anaconda3/envs/PythonData/lib/python3.6/site-packages/urllib3/connectionpool.py:857: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Users/laura/anaconda3/envs/PythonData/lib/python3.6/site-packages/urllib3/connectionpool.py:857: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Users/laura/anaconda3/envs/PythonData/lib/python3.6/site-packages/urllib3/connectionpool.py:857: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Users/laura/anaconda3/envs/PythonData/l

In [61]:
output_dict

{'body_copy': <div class="css-53u6y8"><p class="css-1ygdjhk evys1bk0">MEDELLÍN, Colombia — The prediction was eye-popping even for Venezuela’s collapsing economy: One million percent inflation by year’s end, according to the International Monetary Fund.</p><p class="css-1ygdjhk evys1bk0">On Monday, the fund released a <a class="css-1g7m0tk" href="https://blogs.imf.org/2018/07/23/outlook-for-the-americas-a-tougher-recovery/" rel="noopener noreferrer" target="_blank" title="">periodic estimate of growth and inflation rates</a> around Latin America, a roundup that included neighbors like Chile (where inflation should be headed to 3 percent) and Peru (where growth is predicted to reach 3.7 percent).</p><p class="css-1ygdjhk evys1bk0">But it is Venezuela that continues to chart a different course from neighbors — with an increasingly breakneck economic disintegration.</p><p class="css-1ygdjhk evys1bk0">Real gross domestic product, the value of goods and services produced in the economy and 